In [7]:
###### Plataforma de Métricas (5 anos) #######

import pandas as pd
import numpy as np
import os
# from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

###################################################

file_path = 'results_5_years/nbeats_results_pytorch_5_years.xlsx'
data = pd.read_excel(file_path)

In [8]:
data

,FORECAST_STEPS,TIME_FORECAST,TYPE_PREDICTIONS,STATE,PRODUCT,RMSE,RRMSE,MAPE,PBE,POCID,MASE,PREDICTIONS,ERROR
0,12,12,N-BEATS,pe,etanolhidratado,NaN,NaN,NaN,NaN,NaN,NaN,"[34228.48974740097, 34602.79540011775, 23392.7...",NaN
1,12,12,N-BEATS,pe,gasolinac,NaN,NaN,NaN,NaN,NaN,NaN,"[126395.16000424286, 115412.53934898313, 11038...",NaN
2,12,12,N-BEATS,pe,glp,NaN,NaN,NaN,NaN,NaN,NaN,"[45878.26529672605, 46313.523186033475, 45613....",NaN
3,12,12,N-BEATS,pe,oleodiesel,NaN,NaN,NaN,NaN,NaN,NaN,"[110296.4045946421, 114167.12829249374, 98912....",NaN
4,12,12,N-BEATS,pe,querosenedeaviacao,NaN,NaN,NaN,NaN,NaN,NaN,"[18279.098046951007, 17137.58880317387, 17117....",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,12,12,N-BEATS,ce,etanolhidratado,NaN,NaN,NaN,NaN,NaN,NaN,"[11681.592008113046, 7449.037725613179, 7189.5...",NaN
131,12,12,N-BEATS,ce,gasolinac,NaN,NaN,NaN,NaN,NaN,NaN,"[116780.27215575015, 108028.15126677522, 10996...",NaN
132,12,12,N-BEATS,ce,glp,NaN,NaN,NaN,NaN,NaN,NaN,"[41982.71466827822, 41997.651848267364, 44199....",NaN
133,12,12,N-BEATS,ce,oleodiesel,NaN,NaN,NaN,NaN,NaN,NaN,"[81352.5529812616, 80345.18753351543, 80637.24...",NaN


In [10]:
horizon = 60

###################################################

def rrmse(y_true, y_pred, mean_y_true_serie_completa):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mean_y_true = np.mean(y_true)
    rrmse_value = rmse / mean_y_true_serie_completa
 
    return rrmse_value

def baseline_persistent_window(series, h):
  subsequence = (series[-h*2:-h]).values
  return subsequence

epislon = 1e-20

def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_true-y_pred)/np.sum(y_true))
  else:
    return 100*(np.sum(y_true-y_pred)/(np.sum(y_true) + epislon))
  
def mase(y_true, y_pred, y_baseline):
  mae_pred = np.mean(np.abs(y_true - y_pred))
  mae_naive = np.mean(np.abs(y_true - y_baseline))
  if mae_naive == 0:
    result = mae_pred/(mae_naive + epislon)
  else:
    result = mae_pred/mae_naive
  return result 

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

def save_results(model, mape_result, pocid_result, pbe_result, mase_result, rrmse_result, state, produtct, pontos, filename="Plat_Metricas_5years_Output.csv"):
    data = {
        "TYPE_PREDICTIONS": [model],
        "STATE": state,
        "PRODUCT": produtct,
        "MAPE": [mape_result],
        "POCID": [pocid_result],
        "PBE": [pbe_result],
        "MASE(2020,2021,2022,2023,2024)": [mase_result],
        "RRMSE": [rrmse_result],
        "PONTOS": [pontos]
    }
    df = pd.DataFrame(data)
    # Verificar se o arquivo já existe para evitar reescrever o cabeçalho
    if not os.path.isfile(filename):
        df.to_csv(filename, index=False)  # Primeira gravação com cabeçalho
    else:
        df.to_csv(filename, mode='a', index=False, header=False)  # Adiciona sem o cabeçalho


#######################################

for i in range(1, data.shape[0]):
  data = data.astype(str)

  product = data.iloc[i, 4]  # First cell for product
  state = data.iloc[i, 3]    # Second cell for state
  model = data.iloc[i, 2] # Third cell for model

  print(data["PREDICTIONS"])
  predictions_clean = data.iloc[i]["PREDICTIONS"].strip('[]')  # Remove colchetes
  predictions_list = predictions_clean.split(',')  # Separa por vírgula

  # predictions_clean = data["PREDICTIONS"].strip('[]')  # Remove colchetes
  # predictions_list = predictions_clean.split(',')  # Separa os valores por vírgula

  # Agora, converta os valores limpos para float
  predictions_float = [float(x) for x in predictions_list]  # Converte para float

  # Cria o DataFrame com os valores convertidos
  predictions_df = pd.DataFrame(predictions_float, columns=['Predictions'])
  

  ################ Plataforma de Métricas ##############

  df = pd.read_csv(f"database/venda_process/mensal/uf/{product}/mensal_{state}_{product}.csv", header=0, sep=";")
  df = pd.DataFrame(df)

  last_date = df['timestamp'].tail(horizon)
  last_m3 = df['m3'].tail(horizon)

  new_df = pd.DataFrame({
              'timestamp': last_date,
              'product': product,
              'state': state,
              'm3': last_m3.values})
  new_df = new_df.reset_index(drop=True)

  predictions_df = pd.DataFrame(predictions_float, columns=['Predictions'])

  mape_result = mape(new_df['m3'], predictions_df['Predictions'])
  pocid_result = pocid(new_df['m3'], predictions_df['Predictions']) 
  pbe_result = pbe(new_df['m3'], predictions_df['Predictions'])
  rrmse_result = rrmse(new_df['m3'], predictions_df['Predictions'], df['m3'].mean())

  ##################### NEW MASE #################################

  h=12 # horizonte do baseline

  y_baseline5 = baseline_persistent_window(df, h)
  y_baseline4 = baseline_persistent_window(df.iloc[:-h], h)
  y_baseline3 = baseline_persistent_window(df.iloc[:-h*2], h)
  y_baseline2 = baseline_persistent_window(df.iloc[:-h*3], h)
  y_baseline1 = baseline_persistent_window(df.iloc[:-h*4], h)


  mase5 = mase(new_df['m3'].tail(h), predictions_df['Predictions'].tail(h),y_baseline5[:,1])
  mase4 = mase(new_df['m3'].iloc[-h*2:-h], predictions_df['Predictions'].iloc[-h*2:-h], y_baseline4[:, 1])
  mase3 = mase(new_df['m3'].iloc[-h*3:-h*2], predictions_df['Predictions'].iloc[-h*3:-h*2], y_baseline3[:, 1])
  mase2 = mase(new_df['m3'].iloc[-h*4:-h*3], predictions_df['Predictions'].iloc[-h*4:-h*3], y_baseline2[:, 1])
  mase1 = mase(new_df['m3'].iloc[-h*5:-h*4], predictions_df['Predictions'].iloc[-h*5:-h*4], y_baseline1[:, 1])

  formatted_array = np.array([mase1, mase2, mase3, mase4, mase5])
  formatted_array2 = [f"{value:.4f}" for value in formatted_array]
  mase_result = ', '.join(formatted_array2)

  ############### Métricas ##############

  save_results(model, mape_result, pocid_result, pbe_result, mase_result,rrmse_result, state, product, predictions_df['Predictions'].tolist())

  ############### Plot Predictions ##############

  print(f"MODELO: {model}, MAPE: {mape_result:.5f}, POCID: {pocid_result:.5f}, PBE: {pbe_result:.5f}, MASE: {mase_result}, RRMSE: {rrmse_result:.5f}")

  import matplotlib.pyplot as plt
  import pandas as pd 
  date_range = pd.date_range(start='2019-02-01', periods=len(predictions_df['Predictions']), freq='MS')

  predictions_df = predictions_df.head(len(date_range))

  # plt.figure(figsize=(6, 3)) 
  # plt.plot(date_range, new_df['m3'].tail(horizon), label='Valores Reais', color='blue')
  # plt.plot(date_range, predictions_df['Predictions'], label='Forecast', color='red', linestyle='--')

  # plt.title(f'Product: {product}, Estado: {state}, Modelo: {model}')
  # plt.xlabel(f'{product}')
  # plt.ylabel('Valores em m3')
  # plt.legend()
  # plt.show()


0      [34228.48974740097, 34602.79540011775, 23392.7...
1      [126395.16000424286, 115412.53934898313, 11038...
2      [45878.26529672605, 46313.523186033475, 45613....
3      [110296.4045946421, 114167.12829249374, 98912....
4      [18279.098046951007, 17137.58880317387, 17117....
                             ...                        
130    [11681.592008113046, 7449.037725613179, 7189.5...
131    [116780.27215575015, 108028.15126677522, 10996...
132    [41982.71466827822, 41997.651848267364, 44199....
133    [81352.5529812616, 80345.18753351543, 80637.24...
134    [13927.819166127309, 11811.467412558448, 11188...
Name: PREDICTIONS, Length: 135, dtype: object
MODELO: N-BEATS, MAPE: 0.11047, POCID: 64.40678, PBE: -1.87706, MASE: 0.9801, 1.6138, 1.3608, 1.0102, 0.9200, RRMSE: 0.20200
0      [34228.48974740097, 34602.79540011775, 23392.7...
1      [126395.16000424286, 115412.53934898313, 11038...
2      [45878.26529672605, 46313.523186033475, 45613....
3      [110296.4045946421, 1141